In [3]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Masking
from keras.layers import Dropout
from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd
import argparse
import pickle

VAL_STEPS = None
TRAIN_STEPS = None


class SequenceBatchGenerator(Sequence):
    """
        A generator class to produce a single batch of sequences
        for LSTM training

        Arguments:
            x_set: The whole training set, python list of length m_examples.
            A single example can be accessed in the manner x_set[example_idx]
            and is a numpy array of shape (1, timesteps, n_features). Timesteps
            can vary between examples.

            y_set: The labels corresponding to elements in x_set

            batch_size: The batch size to be used in training

        Outputs:
            batch_x_tensor: Numpy array of shape (batch_size, max_timesteps_batch,
            n_input_features)
            batch_y_tensor: Numpy array of shape (batch_size, max_timesteps_batch,
            n_output_features)


        #https://keras.io/utils/#sequence


    """

    def __init__(self, x_set, y_set, batch_size=128):
        """
        """
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

        # Make sure n_features is same for all examples
        unique_input_dims = len(set([example.shape[2] for example in x_set]))
        unique_output_dims = len(set([example.shape[2] for example in y_set]))
        if not unique_input_dims == unique_output_dims == 1:
            raise Exception("n_features needs to be same for all examples")

        self.input_dim = x_set[0].shape[2]
        self.output_dim = y_set[0].shape[2]


    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size].copy()
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size].copy()

        # get all the stuff required for reshaping
        max_timesteps_batch = max([seq.shape[1] for seq in batch_x])

        # initialize return variables as 3D tensors
        batch_x_tensor = np.zeros((len(batch_x), max_timesteps_batch, self.input_dim))
        batch_y_tensor = np.zeros((len(batch_y), max_timesteps_batch, self.output_dim))

        # Zero pad all samples within batch to max length
        for i in range(len(batch_x)):
            padding_dims = ((0, 0), (0, max_timesteps_batch - batch_x[i].shape[1]), (0, 0))
            batch_x[i] = np.pad(batch_x[i], padding_dims, 'constant', constant_values=(None, -999))
            batch_y[i] = np.pad(batch_y[i], padding_dims, 'constant', constant_values=(None, -999))

            # Reshape to meet Keras expectation
            batch_x[i][0] = np.reshape(batch_x[i].transpose(), (1, max_timesteps_batch, self.input_dim))
            batch_y[i][0] = np.reshape(batch_y[i].transpose(), (1, max_timesteps_batch, self.output_dim))

            # Append x, y to returnable tensor
            batch_x_tensor[i, :, :] = batch_x[i]
            batch_y_tensor[i, :, :] = batch_y[i]

        return batch_x_tensor, batch_y_tensor

def get_model(input_dim, lstm_layers=2, LSTM_units=200, dropout_rate=0.2):
    # Initialize the RNN
    model = Sequential()

    #LSTM Layers and dropout regularization
    model.add(Masking(mask_value=-999.0, input_shape=(None, input_dim)))
    # model.add(LSTM(units = LSTM_units, return_sequences=True,
    #                   input_shape = (None, input_dim)))
    # model.add(Dropout(rate = dropout_rate))

    for i in range(lstm_layers-1):
        model.add(LSTM(units = LSTM_units, return_sequences=True))
        # model.add(Dropout(rate = dropout_rate))

    #Linear output layer
    model.add(TimeDistributed(Dense(1)))

    return model


def get_data_from_pickle(FLAGS):
    return pickle.load(open(FLAGS.file_path, 'rb'))



def train(model, batch_gen, batch_test_gen):
    """

    Args:
        model (keras.models.Sequential:
        batch_gen (SequenceBatchGenerator):
        batch_test_gen (SequenceBatchGenerator:

    Returns:

    """
    # Set up some useful stuff
    from datetime import datetime as dt

    ny = dt.now().strftime("%d-%m-%Y__%H_%M_%S")
    model_outfile = f'{ny}.h5'

    # save_best_only = True --> model_outfile will be overwritten each time val_loss improves
    model_checkpoint = ModelCheckpoint(model_outfile, monitor='val_loss', verbose=1, save_best_only=True)
    csv_logger = CSVLogger(f'{ny}.log')

    # Compile and train the model

    model.compile(optimizer='adam', loss='mean_absolute_error')
    model.fit_generator(batch_gen, epochs=2000, verbose=1, validation_data=batch_test_gen,
                        use_multiprocessing=False, callbacks=[model_checkpoint, csv_logger])


C:\Users\Panu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
import pickle

In [5]:
X_train, X_test, y_train, y_test = pickle.load(open('../data/trips_fullscale_dataset.pickle', 'rb'))

In [6]:
batch_gen = SequenceBatchGenerator(X_train, y_train)


In [7]:
x_tensor, y_tensor = batch_gen.__getitem__(0)

In [10]:
x_tensor[2]

array([[-4.33896593e-01, -1.95278045e-01, -7.59687705e-02,
         2.22665958e-01,  5.09731303e-01,  7.94386360e-01],
       [ 1.11881117e+00,  1.34441417e+00,  3.20282637e-01,
        -1.29991320e+00, -8.62927923e-01,  1.27218539e-01],
       [ 8.06919859e-01,  1.33115758e+00,  1.38563010e+00,
        -2.03955083e-01, -4.13891200e-01,  7.21595651e-01],
       ...,
       [-9.99000000e+02, -9.99000000e+02, -9.99000000e+02,
        -9.99000000e+02, -9.99000000e+02, -9.99000000e+02],
       [-9.99000000e+02, -9.99000000e+02, -9.99000000e+02,
        -9.99000000e+02, -9.99000000e+02, -9.99000000e+02],
       [-9.99000000e+02, -9.99000000e+02, -9.99000000e+02,
        -9.99000000e+02, -9.99000000e+02, -9.99000000e+02]])

In [11]:
y_tensor[2]

array([[-1.31902174e+00],
       [-1.29565334e+00],
       [-1.26868979e+00],
       [-1.23813111e+00],
       [-1.19319186e+00],
       [-1.16622832e+00],
       [-1.13926477e+00],
       [-1.17603324e+00],
       [-1.21280171e+00],
       [-1.20936999e+00],
       [-1.19588822e+00],
       [-1.13926477e+00],
       [-1.06376684e+00],
       [-9.99054333e-01],
       [-1.01792882e+00],
       [-1.06915955e+00],
       [-1.05298142e+00],
       [-9.72090788e-01],
       [-8.96592860e-01],
       [-8.45362123e-01],
       [-8.58843896e-01],
       [-8.85807442e-01],
       [-9.59507800e-01],
       [-1.00444704e+00],
       [-9.99453793e-01],
       [-9.61305369e-01],
       [-9.34341824e-01],
       [-8.75022023e-01],
       [-7.77953259e-01],
       [-6.86277204e-01],
       [-6.05386566e-01],
       [-5.56852184e-01],
       [-5.32584993e-01],
       [-5.29888639e-01],
       [-5.56852184e-01],
       [-6.05386566e-01],
       [-6.50550505e-01],
       [-6.42124397e-01],
       [-6.3